In [1]:
from pd4ml import Airshower
from matplotlib import pyplot as plt 
x,y = Airshower.load_data("train")
toa_train = x['features'][0][::10,:,-1].reshape((-1,9,9,1))
del x
x_test, y_test = Airshower.load_data("test")
toa_test =  x_test['features'][0][::10,:,-1].reshape((-1,9,9,1))
del x_test
y = y[::10]
y_test = y_test[::10]

In [2]:
from keras import layers
from keras.models import Model

input = layers.Input(shape=(9, 9, 1))
x = layers.Conv2D(50, (3, 3), activation="relu", padding="same")(input)
x = layers.MaxPooling2D((2, 2), padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(50)(x)
x = layers.Dense(50)(x)
x = layers.Dense(1)(x)

retino = Model(input, x)
retino.compile(optimizer="adam", loss="mean_squared_error")
retino.summary()

2022-06-21 18:58:16.957588: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-21 18:58:16.957622: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 9, 9, 1)]         0         
                                                                 
 conv2d (Conv2D)             (None, 9, 9, 50)          500       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 5, 5, 50)         0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 1250)              0         
                                                                 
 dense (Dense)               (None, 50)                62550     
                                                                 
 dense_1 (Dense)             (None, 50)                2550      
                                                             

2022-06-21 18:58:20.150678: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-21 18:58:20.150710: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-21 18:58:20.150738: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (djanloo-ratchet): /proc/driver/nvidia/version does not exist
2022-06-21 18:58:20.151473: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
retino.fit(
    x=toa_train,
    y=y,
    epochs=50,
    batch_size=128,
    shuffle=True,
    validation_data=(toa_test, y_test),
)


Epoch 1/50
55/55 [==============================] - 1s 23ms/step - loss: 2936.5356 - val_loss: 2883.7493
Epoch 2/50
55/55 [==============================] - 1s 17ms/step - loss: 2923.5259 - val_loss: 2907.3232
Epoch 3/50
55/55 [==============================] - 1s 15ms/step - loss: 2924.5906 - val_loss: 2882.4243
Epoch 4/50
55/55 [==============================] - 1s 15ms/step - loss: 2919.9844 - val_loss: 2881.4021
Epoch 5/50
55/55 [==============================] - 1s 15ms/step - loss: 2924.9990 - val_loss: 2882.6179
Epoch 6/50
55/55 [==============================] - 1s 15ms/step - loss: 2912.7993 - val_loss: 2872.1453
Epoch 7/50
55/55 [==============================] - 1s 15ms/step - loss: 2939.8623 - val_loss: 2883.9990
Epoch 8/50
55/55 [==============================] - 1s 17ms/step - loss: 2908.9543 - val_loss: 2914.6406
Epoch 9/50
55/55 [==============================] - 1s 20ms/step - loss: 2912.0994 - val_loss: 2877.7327
Epoch 10/50
55/55 [==============================] - 1s

In [26]:
import numpy as np
errors = np.abs( retino.predict(toa_test).squeeze() - y_test ) / y_test * 100 
print(np.mean(errors))

94/94 [==============================] - 1s 6ms/step
5.322098825477716
